In [1]:
from credmark.cmf.engine.model_loader import ModelLoader
from credmark.cmf.engine.context import EngineModelContext

from credmark.dto import EmptyInput

from credmark.cmf.types import Contract, Token

from credmark.cmf.types.ledger import (
    BlockTable, ContractTable,
    LogTable, ReceiptTable, TokenTable, TokenTransferTable,
    TraceTable, TransactionTable, LedgerTable,
    LedgerAggregate, LedgerModelOutput
)
import sys
sys.tracebacklimit=0

In [2]:
model_loader = ModelLoader(['../models'], None, True)

params = {'chain_id': 1,
          'block_number': None,
          'model_loader': model_loader,
          'chain_to_provider_url': {'1': 'https://eth-mainnet.alchemyapi.io/v2/4HREehwJhXq-fkm1llCDGol-swXEXj0L'},
          'api_url': 'http://h2.runchee.com:18700',
          'run_id': None,
          'console': True,
          'use_local_models': '*',
         }
context = EngineModelContext.create_context(**params)

In [3]:
context.block_number

14879234

In [4]:
test_cases = [
    {'base': 'ETH', 'quote': 'ETH'},
    {'base': 'ETH', 'quote': 'USD'},
    {'base': 'ETH', 'quote': 'GBP'},    
    {'base': 'ETH', 'quote': 'CNY'},
    {'base': 'CNY', 'quote': 'USD'},    
    {'base': 'USD', 'quote': 'ETH'},
    {'base': 'GBP', 'quote': 'ETH'},
    {'base': 'BTC', 'quote': 'CNY'},
    {'base': {'address': '0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E'}, 'quote': 'USD'}, # ILV
    {'base': 'JPY', 'quote': {'address': '0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E'}},
    {'quote': 'JPY', 'base': {'address': '0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E'}},
    {'base': 'JPY', 'quote': {'address':'0x85f138bfEE4ef8e540890CFb48F620571d67Eda3'}}, # WAVAX
    {'base': 'USD', 'quote': {'address': '0x85f138bfEE4ef8e540890CFb48F620571d67Eda3'}},
    {'quote': 'USD', 'base': {'address': '0x85f138bfEE4ef8e540890CFb48F620571d67Eda3'}},
    {'quote': 'JPY', 'base': {'address': '0x85f138bfEE4ef8e540890CFb48F620571d67Eda3'}},
    {'base': 'JPY', 'quote': {'address': '0x85f138bfEE4ef8e540890CFb48F620571d67Eda3'}},
    {'base': 'CNY', 'quote': {'symbol':'USDC'}},
    {'quote': 'CNY', 'base': {'symbol':'DAI'}},
    {'base': 'ETH', 'quote': {'address':'0xc00e94Cb662C3520282E6f5717214004A7f26888'}}, # COMP
    {'base': 'CNY', 'quote': {'address':'0xc00e94Cb662C3520282E6f5717214004A7f26888'}},
    {'quote': 'CNY', 'base': {'address':'0xc00e94Cb662C3520282E6f5717214004A7f26888'}},
    {'base': 'CNY', 'quote': {'address':'0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E'}}, # ILV
    {'quote': 'CNY', 'base': {'address':'0x767FE9EDC9E0dF98E07454847909b5E959D7ca0E'}},
]

for case in test_cases:
    price = context.models.price.oracle_chainlink(case)
    print(f"{case['base']}/{case['quote']}: {price['price']}")

ETH/ETH: 1.0
ETH/USD: 1952.41
ETH/GBP: 1551.6991988809766
ETH/CNY: 13016.153441022941
CNY/USD: 0.149999
USD/ETH: 0.0005121875016005859
GBP/ETH: 0.0006444548020139212
BTC/CNY: 208820.71735092037


ModelInputError: Error validating model price.oracle-chainlink input: 1 validation error for PriceInput
base
  Address instance must be created with a string or int (type=type_error)

# Documentation

The input to model `price.oracle-chainlink` is defined below

```
class PriceInput(DTO):
    """
    In FX, the pair is quoted as base/quote for 1 base = x quote
    e.g. 1883.07 ETH / USD means 1883.07 USD for 1 ETH.

    *Base* token to get the value in the quote token
    *Quote* token to determine the value of the base token.

    If quote is not provided, default to the native token of the chain, i.e. ETH for Ethereum.

    Fiat is expressed in the currency code in ISO 4217.

    Base and quote can be either Token (symbol or address)
    or code (BTC, ETH, and all fiat, like USD, EUR, CNY, etc.)

    For fiat, with USD being the most active traded currency.
    It's direct quoting with (x USD/DOM) for x DOM = 1 USD, e.g. USD/JPY;
    and indirect quoting with (x DOM/USD) for x USD = 1 DOM, e.g. GBP/USD.

    For DeFi, we call it a direct quoting when the native token is the base.
    e.g. ETH / USD
    """
    base: TokenOrCode = DTOField(description='Base token to get the value for')
    quote: Union[None, TokenOrCode] = DTOField(None, description='Quote token to count the value')
```

Code are sources from Chainlink's documentation https://docs.chain.link/docs/ethereum-addresses/

```
class TokenOrCode(Token):
    TRIPLET_CONVERSION: ClassVar[Dict[str, Address]] = {
        # 0x0000000000000000000000000000000000000348
        'USD': to_hex_address(840),
        # 0x000000000000000000000000000000000000033a
        'GBP': to_hex_address(826),
        # 0x00000000000000000000000000000000000003d2
        'EUR': to_hex_address(978),
        'JPY': to_hex_address(392),
        'CNY': to_hex_address(156),
        'AUD': to_hex_address(36),
        'KRW': to_hex_address(410),
        'BRL': to_hex_address(986),
        'CAD': to_hex_address(124),
        'CHF': to_hex_address(756),
        'IDR': to_hex_address(360),
        'INR': to_hex_address(356),
        'NGN': to_hex_address(566),
        'NZD': to_hex_address(554),
        'PHP': to_hex_address(608),
        'SGD': to_hex_address(702),
        'TRY': to_hex_address(949),
        'ZAR': to_hex_address(710),
        'ETH': Address('0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE'),
        'BTC': Address('0xbBbBBBBbbBBBbbbBbbBbbbbBBbBbbbbBbBbbBBbB'),
    }

    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, val):
        if isinstance(val, str):
            tok = Token(address=cls.TRIPLET_CONVERSION.get(val.upper(), None))
            if tok is None:
                raise ModelDataError(f'Unsupported currency code {val}')
            return tok
        else:
            return Token.validate(val)

```